# Lab 4. Energy Agent Collaborator

## Introduction

In this notebook we show you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. This composable pattern allows you to build a chain of agents, as shown in the figure below.

![collaboration_hierarch](img/multi-agent_flow.png)

In this lab, we will create a supervisor agent that interacts with the subagents created in the previous lab.

The following represents the complete architecture of multi-agent collaborator.

![Architecture](img/energy_manager_agent.png)

This Lab will is divided into two notebooks:

- [4.1_energy_agent_collaborator.ipynb](4.1_energy_agent_collaborator.ipynb): Which contains Agent Setup
- [4.2_energy_agent_invocation.ipynb](4.2_energy_agent_invocation.ipynb): Which contains Agent Invocation

## Setup

Make sure that your boto3 version is the latest one.

If not, return no [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run Setup block again.

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [ ]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [ ]:
energy_agent_name = f"energy-agent-{agent_suffix}"

energy_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{energy_agent_name}'

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `create_agent`: Create a new agent and respective IAM roles
- `associate_agents`: Associate sub-agents with multi-agent collaborator

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

Create the energy agent

In [ ]:
energy_agent = agents.create_agent(
    energy_agent_name,
    """
        You are a energy helper bot. 
        You can help customers with operations related with their energy, like consumption, forecast, peak usage, etc. 
    """,
    """
        You are a energy helper bot.
        You can retrieve energy consumption and forecast for a specific users and help them to be compliant with energy rules.
        You can also retrieve solar panel information and solar panel ticket for a specific users and help them to be compliant with energy rules.
        You can also get current information about peaks and can redistribute load.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 
    """,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR_ROUTER'
)

energy_agent

Store environment variables to be used on next notebooks.

In [ ]:
energy_agent_id = energy_agent[0]
%store energy_agent_id
%store energy_agent_name

### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside energy one (which is multi-agent collaborator).

In [ ]:
%store -r
forecast_agent_alias_arn, solar_agent_alias_arn, peak_agent_alias_arn

In [ ]:
# If store fails, add Alias Arn here
#forecast_agent_alias_arn = ''
#compliance_agent_alias_arn = ''
#consumption_agent_alias_id = ''

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': forecast_agent_alias_arn,
        'sub_agent_instruction': """Delegate energy consumption analysis and forecasting tasks to the Forecasting Agent, ensuring adherence to its specific protocols and capabilities.""",
        'sub_agent_association_name': 'ForecastCoordinationAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': solar_agent_alias_arn,
        'sub_agent_instruction': """Assign solar panel-related inquiries and issues to the Solar Panel Agent, respecting its scope and support ticket protocol.""",
        'sub_agent_association_name': 'SolarSupportManagementAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': peak_agent_alias_arn,
        'sub_agent_instruction': """Direct peak load management and energy optimization tasks to the Peak Load Manager Agent, leveraging its analytical capabilities.""",
        'sub_agent_association_name': 'PeakLoadOptimizationAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Check if parameters are correclty formated

In [ ]:
sub_agents_list

Associate all sub-agents with this one (main one).

In [ ]:
energy_agent_alias_id, energy_agent_alias_arn = agents.associate_sub_agents(
    energy_agent_id, sub_agents_list
)

Check response return, to see all returned parameteres:

In [ ]:
energy_agent_alias_id, energy_agent_alias_arn

In [ ]:
%store energy_agent_alias_id
%store energy_agent_alias_arn

## Next Steps
Congratulations! We've now created a supervisor agent. Next let's invoke our agent